# Model zoo

In [ ]:
import torch
import numpy as np
import tensorflow as tf

## Generate toy data

In [ ]:
def generate_data(n=16, samples_per_class=1000):
    """
    Generate some classification data
    
    Args:
        n (int): square root of the number of features.
        samples_per_class (int): number of samples per class.
    
    Returns:
        a tuple containing data and labels.
    """
    # data for a class
    a_class_samples = np.random.rand(samples_per_class, n, n).astype(np.float32)
    a_class_labels = np.zeros(samples_per_class, dtype=int)
    # data for another class
    another_class_samples = np.array([
        np.eye(n)*np.random.rand(1).item()
        for _ in range(samples_per_class)
    ]).astype(np.float32)
    another_class_labels = np.ones(samples_per_class, dtype=int)
    # aggregate data
    data = np.vstack([a_class_samples, another_class_samples])
    labels = np.hstack([a_class_labels, another_class_labels])
    # prepare a shuffled index
    indices = np.arange(data.shape[0])
    np.random.shuffle(indices)
    return data[indices], labels[indices]

In [ ]:
# get data
n = 16
features = n*n
number_of_classes = 2
X_train, y_train = generate_data(n=n)
X_test, y_test = generate_data(n=n)

## MLP

In [ ]:
# parameters
units = [32, 8]

### PyTorch

In [ ]:
class MLP(torch.nn.Module):
    """A MultiLayer Perceptron class."""
    
    def __init__(
        self, features,
        units=[8], number_of_classes=2,
        activation_module=torch.nn.ReLU
    ):
        """
        Inititalize the MLP.
        
        Args:
            features (int): number of features.
            units (list): list of hidden layer units.
            number_of_classes (int): number of classes to predict.
            activation_module (torch.nn.Module): module representing
                the activation function to apply in the hidden layers.
        """
        super(MLP, self).__init__()
        self.units = [features] + units
        self.activation_module = activation_module
        self.hidden_layers = torch.nn.Sequential(*[
            torch.nn.Sequential(
                torch.nn.Linear(input_size, output_size),
                self.activation_module()
            )
            for input_size, output_size in zip(
                self.units, self.units[1:]
            )
        ])
        self.last_layer = self.last_layer = torch.nn.Sequential(*[
            torch.nn.Linear(self.units[-1], number_of_classes),
            torch.nn.Softmax(dim=1)
        ])
    
    def forward(self, sample):
        """
        Apply the forward pass of the model.

        Args:
            sample (torch.Tensor): a torch.Tensor representing a sample.
        Returns:
            a torch.Tensor containing softmaxed predictions.
        """
        encoded_sample =  self.hidden_layers(sample)
        return self.last_layer(encoded_sample)

In [ ]:
X = torch.from_numpy(X_train.reshape(-1, features))
model = MLP(features=features, units=units, number_of_classes=number_of_classes)
model(X)

### TensorFlow/Keras

In [ ]:
def mlp(
    features,
    units=[8], number_of_classes=2,
    activation='relu'
):
    """
    Build a MLP.

    Args:
        features (int): number of features.
        units (list): list of hidden layer units.
        number_of_classes (int): number of classes to predict.
        activation (str): string identifying the activation used.
    
    Returns:
        a tf.keras.Model.
    """
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units[0], activation=activation, input_shape=(features,)))
    for unit in units[1:]:
        model.add(tf.keras.layers.Dense(unit, activation=activation))
    model.add(tf.keras.layers.Dense(number_of_classes, activation='softmax'))
    return model

In [ ]:
X = X_train.reshape(-1, features)
model = mlp(features=features, units=units, number_of_classes=number_of_classes)
model.predict(X)

## AE

In [ ]:
# parameters
units = [32, 8]

### PyTorch

In [ ]:
class AE(torch.nn.Module):
    """An AutoEncoder class."""
    
    def __init__(
        self, features,
        units=[8], activation_module=torch.nn.ReLU
    ):
        """
        Inititalize the AE.
        
        Args:
            features (int): number of features.
            units (list): list of hidden layer units.
            activation_module (torch.nn.Module): module representing
                the activation function to apply in the hidden layers.
        """
        super(AE, self).__init__()
        self.units = [features] + units
        self.activation_module = activation_module
        zipped_units = list(zip(
            self.units, self.units[1:]
        ))
        # encoding
        self.encoder = torch.nn.Sequential(*[
            torch.nn.Sequential(
                torch.nn.Linear(input_size, output_size),
                self.activation_module()
            )
            for input_size, output_size in zipped_units
        ])
        # decoding
        last_decoder_units, *hidden_decoder_units = zipped_units
        self.decoder = torch.nn.Sequential(*[
            torch.nn.Sequential(
                torch.nn.Linear(input_size, output_size),
                self.activation_module()
            )
            for input_size, output_size in map(
                lambda t: t[::-1],
                hidden_decoder_units[::-1]
            )
        ])
        self.last_layer = torch.nn.Linear(*last_decoder_units[::-1])
    
    def forward(self, sample):
        """
        Apply the forward pass of the model.

        Args:
            sample (torch.Tensor): a torch.Tensor representing a sample.
        Returns:
            a torch.Tensor containing the reconstructed example.
        """
        encoded_sample = self.encoder(sample)
        decoded_sample = self.decoder(encoded_sample)
        return self.last_layer(decoded_sample)

In [ ]:
X = torch.from_numpy(X_train.reshape(-1, features))
model = AE(features=features, units=units)
model(X)

In [ ]:
# get encoded representation
model.encoder(X)

### TensorFlow/Keras

In [ ]:
def ae(features, units=[8], activation='relu'):
    """
    Build an AE.

    Args:
        features (int): number of features.
        units (list): list of hidden layer units.
        number_of_classes (int): number of classes to predict.
        activation (str): string identifying the activation used.
    
    Returns:
        a tf.keras.Model.
    """
    model = tf.keras.Sequential()
    # encoding
    model.add(tf.keras.layers.Dense(
        units[0], activation=activation, input_shape=(features,)
    ))
    for unit in units[1:]:
        model.add(tf.keras.layers.Dense(unit, activation=activation))
    # decoding
    for unit in units[::-1][1:]:
        model.add(tf.keras.layers.Dense(unit, activation=activation))
    model.add(tf.keras.layers.Dense(features))
    return model

In [ ]:
X = X_train.reshape(-1, features)
model = ae(features=features, units=units)
model.predict(X)

In [ ]:
# get encoded representation
encoder = tf.keras.Model(
    inputs=model.input,
    outputs=model.layers[len(units) - 1].output
)
encoder.predict(X)

## CNN

In [ ]:
# parameters
filters = [64, 32]
kernel_size = (3, 3)
channels = 1

### PyTorch

In [ ]:
class CNN(torch.nn.Module):
    """A Convolutional Neural Network class."""
    
    def __init__(
        self, channels,
        filters=[8], kernel_size=(3,3),
        number_of_classes=2,
        activation_module=torch.nn.ReLU
    ):
        """
        Inititalize the CNN.
        
        Args:
            channels (int): number of input channels.
            filters (list): list of filters.
            kernel_size (tuple): size of the kernel.
            number_of_classes (int): number of classes to predict.
            activation_module (torch.nn.Module): module representing
                the activation function to apply in the hidden layers.
        """
        super(CNN, self).__init__()
        self.filters = [channels] + filters
        self.kernel_size = kernel_size
        self.activation_module = activation_module
        self.stacked_convolutions = torch.nn.Sequential(*[
            torch.nn.Sequential(
                torch.nn.Conv2d(input_size, output_size, kernel_size),
                self.activation_module(),
                torch.nn.MaxPool2d((2,2), stride=2)
            )
            for input_size, output_size in zip(
                self.filters, self.filters[1:]
            )
        ])
        self.last_layer = torch.nn.Sequential(*[
            torch.nn.Linear(self.filters[-1], number_of_classes),
            torch.nn.Softmax(dim=1)
        ])
    
    def forward(self, sample):
        """
        Apply the forward pass of the model.

        Args:
            sample (torch.Tensor): a torch.Tensor representing a sample.
        Returns:
            a torch.Tensor containing softmaxed predictions.
        """
        encoded_sample = self.stacked_convolutions(sample)
        return self.last_layer(encoded_sample.mean((2,3)))

In [ ]:
X = torch.from_numpy(np.expand_dims(X_train, 1))
model = CNN(
    channels=channels, filters=filters,
    kernel_size=kernel_size,
    number_of_classes=number_of_classes
)
model(X)

### TensorFlow/Keras

In [ ]:
def cnn(
    channels, input_shape,
    filters=[8], kernel_size=(3,3),
    number_of_classes=2, activation='relu'):
    """
    Build a CNN.

    Args:
        channels (int): number of input channels.
        input_shape (tuple): input shape.
        filters (list): list of filters.
        kernel_size (tuple): size of the kernel.
        number_of_classes (int): number of classes to predict.
        activation (str): string identifying the activation used.
    
    Returns:
        a tf.keras.Model.
    """
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(
        filters[0], kernel_size, activation=activation,
        input_shape=input_shape
    ))
    for a_filter in filters[1:]:
        model.add(tf.keras.layers.Conv2D(
            a_filter, kernel_size, activation=activation
        ))
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(tf.keras.layers.Dense(number_of_classes, activation='softmax'))
    return model

In [ ]:
X = np.expand_dims(X_train, 3)
model = cnn(
    channels=channels, input_shape=X.shape[1:],
    filters=filters, kernel_size=kernel_size,
    number_of_classes=number_of_classes
)
model.predict(X)

## RNN

In [ ]:
# parameters
units = [32, 8]

### PyTorch

In [ ]:
class RNN(torch.nn.Module):
    """A Recurrent Neural Network class."""
    
    def __init__(
        self, input_size, units=[8],
        number_of_classes=2, rnn_cell=torch.nn.GRU
    ):
        """
        Inititalize the RNN.
        
        Args:
            input_size (int): size of the input.
            units (list): list of hidden layer units.
            number_of_classes (int): number of classes to predict.
            rnn_cell (torch.nn.RNNBase): a RNN cell.
        """
        super(RNN, self).__init__()
        self.units = [input_size] + units
        self.rnn_layers = [
            rnn_cell(input_size, output_size)
            for input_size, output_size in zip(
                self.units, self.units[1:]
            )
        ]
        self.last_layer = torch.nn.Sequential(*[
            torch.nn.Linear(self.units[-1], number_of_classes),
            torch.nn.Softmax(dim=1)
        ])
    
    def forward(self, sample):
        """
        Apply the forward pass of the model.

        Args:
            sample (torch.Tensor): a torch.Tensor representing a sample.
        Returns:
            a torch.Tensor containing softmaxed predictions.
        """
        encoded_sample = sample
        for rnn_layer in self.rnn_layers[:-1]:
            encoded_sample, _ = rnn_layer(encoded_sample)
        encoded_sample = self.rnn_layers[-1](encoded_sample)[1].squeeze(0)
        return self.last_layer(encoded_sample)

In [ ]:
X = torch.from_numpy(X_train.transpose((1,0,2)))
model = RNN(
    input_size=n, units=units,
    number_of_classes=number_of_classes
)
model(X)

### TensorFlow/Keras

In [ ]:
def rnn(
    sequence_length, input_size,
    units=[8], number_of_classes=2,
    rnn_cell=tf.keras.layers.GRU
):
    """
    Build a RNN.

    Args:
        sequence_length (int): length of the sequence.
        input_size (int): size of the input.
        units (list): list of hidden layer units.
        number_of_classes (int): number of classes to predict.
        rnn_cell (tf.keras.layers.RNN): a RNN cell.

    Returns:
        a tf.keras.Model.
    """
    model = tf.keras.Sequential()
    is_stacked = len(units) > 1
    model.add(rnn_cell(units=units[0], input_shape=(16, 16,), return_sequences=is_stacked))
    for unit in units[1:-1]:
        model.add(rnn_cell(units=unit, return_sequences=True))
    if is_stacked:
        model.add(rnn_cell(units=units[-1]))
    model.add(tf.keras.layers.Dense(number_of_classes, activation='softmax'))
    return model

In [ ]:
X = X_train
model = rnn(
    sequence_length=n, input_size=n, units=units,
    number_of_classes=number_of_classes
)
model.predict(X)